In [18]:
import math
import numpy as np
import pandas as pd

In [19]:
X = pd.read_csv("Data/BreastCancerDiagnostic.csv", header=None)
Y = X.iloc[:,-1:]
X = X.iloc[:, :-1] # The last column is dependent variable
X = X.to_numpy()
Y = Y.to_numpy()

In [20]:
assert(X.shape == (569, 31))
assert(X[0,0] == 842302)
assert(X[0,1] == 17.99)
assert(X[0,2] == 10.38)
assert(X[0,3] == 122.80)
assert(np.sum(X) == 17282628559.459633)

In [21]:
def norm_variance(X):
    return (X - X.mean(axis=0)) / X.std(axis=0)

def norm_scale01(X):
    mi = np.min(X, axis=0)
    ma = np.max(X, axis=0)
    return (X - mi) / (ma - mi);

In [22]:
X1 = norm_variance(X)
assert(math.isclose(X1[:,0][0], -0.23640516608195175))
assert(math.isclose(X1[:,0][1], -0.23640344485199363))
assert(math.isclose(X1[:,0][2],  0.4317410864880675))
assert(math.isclose(X1[:,0][3],  0.4321205416394822))
assert(math.isclose(X1[:,0][4],  0.4322014074246301))
assert(math.isclose(X1[:,0][5], -0.23639328559238046))

X2 = norm_scale01(X1)
assert(math.isclose(X2[:,0][0],  0.000914760426374012))
assert(math.isclose(X2[:,0][1],  0.000914996350008985))
assert(math.isclose(X2[:,0][2],  0.0924954884158686))
assert(math.isclose(X2[:,0][3],  0.0925474991528476))
assert(math.isclose(X2[:,0][4],  0.0925585831744144))
assert(math.isclose(X2[:,0][5],  0.000916388848114947))

In [23]:
assert(math.isclose(X2[:,0][0],  0.000914760426374012)) # First row (sample)
assert(math.isclose(X2[:,0][1],  0.000914996350008985))
assert(math.isclose(X2[:,0][2],  0.0924954884158686))
assert(math.isclose(X2[:,0][3],  0.0925474991528476))
assert(math.isclose(X2[:,0][4],  0.0925585831744144))
assert(math.isclose(X2[:,0][5],  0.000916388848114947))

In [24]:
Y_ = Y.transpose()
for i in range(1, 19):
    assert(Y_[0][i] == 1)
for i in range(19,22):
    assert(Y_[0][i] == -1)
for i in range(22,26):
    assert(Y_[0][i] == 1)

In [25]:
d = X1.shape[1]
px = 1
gap = 0
N = X1.shape[0]
imgSz = ((px*d+gap*(d+1)), (px*d+gap*(d+1)))
maxBarHeight = imgSz[0] - 2*px

In [26]:
imgageGPU = np.zeros((imgSz[0], imgSz[1], N))
assert(imgageGPU.shape == (31, 31, 569))

In [27]:
X2[0,:]

array([0.00091476, 0.52103744, 0.0226581 , 0.54598853, 0.36373277,
       0.59375282, 0.7920373 , 0.70313964, 0.73111332, 0.68636364,
       0.60551811, 0.35614702, 0.12046941, 0.3690336 , 0.27381126,
       0.15929565, 0.35139844, 0.13568182, 0.30062512, 0.31164518,
       0.18304244, 0.62077552, 0.14152452, 0.66831017, 0.45069799,
       0.60113584, 0.61929156, 0.56861022, 0.91202749, 0.59846245,
       0.41886396])

In [28]:
for i in range(N):
    barI = np.floor(maxBarHeight * X2[i,:]).astype(int)
    k = 1
    j = gap + 1
    # Upside down images will be created
    while j < imgSz[1]-gap:
        for i in range(barI[k]):
            imgageGPU[i, j+px-1, i] = 1
        j += (gap + px)

In [29]:
imgageGPU.shape

(31, 31, 569)